In [1]:
from Bio import Entrez
from Bio import SeqIO
Entrez.email = "A.N.Other@example.com"

In [14]:
gene_id = '1007358254'
handle = Entrez.efetch(id=gene_id, db='nucleotide', rettype='gb', retmode='text')

In [15]:
res = SeqIO.read(handle, format='gb')

In [16]:
res

SeqRecord(seq=Seq('GGGATCAGAAAACTCGCTGGTTAATGAGTGGCCAAGAACATATCAGCTTAAGTT...CAA', IUPACAmbiguousDNA()), id='NM_001321072.1', name='NM_001321072', description='Homo sapiens cystathionine-beta-synthase (CBS), transcript variant 5, mRNA', dbxrefs=[])